In [6]:
import random
from datetime import date as dd

import ipywidgets as iw
import matplotlib.colors as mcolors
import pandas as pd
from IPython.core.display import display
from ipywidgets import interact, widgets

from patterns import FileManager, Publisher, df_not_empty


In [13]:
df = pd.DataFrame()
df.index.name = '№'

file_manager, publisher = FileManager(), Publisher()

file_uploader = iw.FileUpload(accept='.json,.csv', multiple=False)
load_button = iw.Button(description='Загрузить данные')
hbox = iw.HBox((file_uploader, load_button))

colors = []
cl_options = {
    'style': {'description_width': '150px'},
    'layout': {'width': '70%'}
}

def on_click(b):
    global df, colors
    [content], [metadata] = file_uploader.data or [''], file_uploader.metadata or [{'type': ''}]
    df = file_manager.load_data(content, metadata)
    colors = random.choices(list(mcolors.CSS4_COLORS.values()), k=len(df)+1)
    
publisher.register((load_button, on_click, 'click'))
load_button.on_click(publisher)
    
display(hbox)

In [10]:
@df_not_empty(target=df)
def run_statistics_module():
    @interact
    def apply_filter(project_name=iw.Text(description='Название проекта: ', **cl_options), 
                     task_name=iw.Text(description='Название задачи: ', **cl_options), 
                     task_id=iw.IntText(description='Номер задачи: ', min=0, **cl_options), 
                     hour_effort=iw.IntText(description='Трудоёмкость: ', min=0, **cl_options), 
                     executor=iw.Text(description='Исполнитель: ', **cl_options),
                     allow_unassigned_tasks=iw.Checkbox(description='Показывать не назначенные задачи', **cl_options),
                     column=iw.Dropdown(description='Поле сортировки: ', options=['', *df.columns], **cl_options),
                     order_type=iw.Dropdown(description='Тип сортировки: ', options=['<', '>'], **cl_options)):
        
        project_name_filter = df['название проекта'].str.startswith(project_name)
        task_name_filter = df['название задачи'].str.startswith(task_name)
        task_id_filter = df['id задачи'] == task_id if task_id != 0 else True
        hour_effort_filter =  df['трудоёмкость'] == hour_effort if hour_effort != 0 else True
        executor_filter_values = [executor, '-'] if allow_unassigned_tasks else [executor]
        executor_filter = df['исполнитель'].str.contains('|'.join(executor_filter_values))
        
        filtered_df = df[project_name_filter & task_name_filter & task_id_filter & hour_effort_filter & executor_filter]
            
        if filtered_df.empty:
            return filtered_df
        
        pie_labels = filtered_df['название проекта'].str.cat(filtered_df['название задачи'], sep =" ")
        pie_plot = filtered_df['трудоёмкость'].plot.pie(autopct='%1.1f%%',
                                                        radius=2, 
                                                        pctdistance=0.8, 
                                                        labeldistance=1.1,
                                                        labels=pie_labels,
                                                        colors=colors)
        pie_plot.set_ylabel('')
        pie_plot.set_title('Отношение трудоёмкости задач', pad=100)
    
        executors = filtered_df.groupby('исполнитель')['исполнитель'].count().reset_index(name='кол-во исполнителей')
        hist_plot = filtered_df.merge(executors, 
                                 left_on='исполнитель', 
                                 right_on='исполнитель').plot.hist(x='кол-во исполнителей', 
                                                                   y='трудоёмкость', 
                                                                   title='Изменение трудоёмкости к кол-ву исполнителей')
        hist_plot.set_xlabel('трудоёмкость')
        hist_plot.set_ylabel('кол-во исполнителей')
        hist_plot.set_yticks(list(executors['кол-во исполнителей']) + [executors['кол-во исполнителей'].max() + 1])
        
        return filtered_df.sort_values(by=column, ascending=order_type == '<') if column else filtered_df
    
run_statistics_module()
    

interactive(children=(Text(value='', description='Название проекта: ', layout=Layout(width='70%'), style=Descr…

In [9]:
@df_not_empty(target=df)
def run_creating_report_module():
    controls = {
        'project_name': iw.Dropdown(description='Название проекта: ', options=df['название проекта'].unique(), **cl_options),
        'task_name': iw.Dropdown(description='Название задачи: ', options=df['название задачи'].unique(), **cl_options),
        'task_id': iw.Dropdown(description='Номер задачи: ', options=df['id задачи'].unique(), **cl_options), 
        'date': iw.DatePicker(description='Дата: ', value=dd.today(), **cl_options), 
        'active_time': iw.FloatSlider(description='Временные затраты: ', min=0, **cl_options),
        'show_all': iw.Checkbox(description='Показывать все элементы', **cl_options)
    }
    widgets.jslink((controls['task_name'], 'index'), (controls['task_id'], 'index'))
    
    create_report_button = iw.Button(description='Создать отчёт')
    report_name_field = iw.Text(description='Имя файла', continuous_update=False)
    report_file_type_dropdown = iw.Dropdown(description='Тип файла', options=['csv', 'json'])
    
    r_df = df.copy()
    r_df['дата'] = dd.today()
    r_df['временные затраты'] = 0.0
    r_df['остаток трудозатрат'] = r_df['трудоёмкость'].astype(float)
    
    def validate_filename_field(change):
        if '/' in change['new'] or '\\' in change['new']:
            report_name_field.value = change['old']
    
    def update_tasks(event):
        target = r_df[r_df['название проекта'] == controls['project_name'].value]
        controls['task_name'].options = target['название задачи']
        controls['task_id'].options = target['id задачи']
    
    def update_task_bound_info(event):    
        condition = (r_df['название проекта'] == controls['project_name'].value) & \
                    (r_df['название задачи'] == controls['task_name'].value) & \
                    (r_df['id задачи'] == controls['task_id'].value)
        target = r_df[condition].iloc[0]

        controls['active_time'].max = float(target['трудоёмкость'])
        controls['date'].value = target['дата']
        controls['active_time'].value = target['временные затраты']
    
    def create_report(project_name, task_name, task_id, date, active_time, show_all):
        condition = (r_df['название проекта'] == project_name) & \
                    (r_df['название задачи'] == task_name) & \
                    (r_df['id задачи'] == task_id)
        columns = ['дата', 'временные затраты', 'остаток трудозатрат']
        
        r_df.loc[condition, columns] = [date, active_time, r_df.loc[condition, 'трудоёмкость'] - active_time]        
        return r_df if show_all else r_df[condition]
    
    def on_click(b):
        file_type = report_file_type_dropdown.value
        path = f'output/{report_name_field.value}.{file_type}'

        file_manager.save_data(r_df, path)
    
    publisher.register((report_name_field, validate_filename_field, 'change'))
    publisher.register((controls['project_name'], update_tasks, 'change'))
    publisher.register((controls['task_id'], update_task_bound_info, 'change'))
    publisher.register((create_report_button, on_click, 'click'))
    
    report_name_field.observe(publisher, 'value')
    controls['project_name'].observe(publisher, 'value')
    controls['task_id'].observe(publisher, 'value')
    create_report_button.on_click(publisher)
    
    hbox = iw.HBox((report_name_field, report_file_type_dropdown, create_report_button))
    
    interact(create_report, **controls)
    display(hbox)
    
    update_tasks(None)
    update_task_bound_info(None)


run_creating_report_module()


interactive(children=(Dropdown(description='Название проекта: ', layout=Layout(width='70%'), options=('Project…